# Compute Optimal Classifier Populations for Boolean Functions

Implements the pipeline to compute optimal classifier populations for Boolean functions. 

In [ ]:
import os
from pathlib import Path
from boolean_functions import *
from xcslib import generate_all_bitstrings

# Generation of Optimal Classifier Populations

## Parameters

In [2]:
# path to the espresso executable
espresso_executable = "../espresso-logic-master/bin/espresso"

# espresso option: guarantees minimum number of product terms, 
# and heuristically minimizes number of literals. Potentially, expensive.
espresso_options = "-Dexact"

In [3]:
Path("./pla/").mkdir(parents=True, exist_ok=True)
Path("./populations/").mkdir(parents=True, exist_ok=True)

## Espresso Table Generator

In [4]:
def generate_action_table_v2(n, binary_function, action, filename):
    inputs = generate_all_bitstrings(n)
    
    with open(filename,"w") as OUTPUT:
        OUTPUT.write(".i %d\n.o 2\n"%(n+1))

        minterms = 0
        for input in inputs:
            output = binary_function(input)
            if (output ==action):
                OUTPUT.write(input+str(action)+" "+"10\n")
            elif (output ==(1-action)):
                OUTPUT.write(input+str(action)+" "+"01\n")
            else:
                raise Exception("Binary function did not output 0 nor 1")
        OUTPUT.write(".e\n")

In [5]:
def simplify_table(input_pla_filename, output_pla_filename, espresso_executable = espresso_executable, espresso_options = espresso_options):
    minimization_command = "%s %s %s > %s"%(espresso_executable,espresso_options,input_pla_filename, output_pla_filename)
    
    minimization_result = os.system(minimization_command)
    if (minimization_result!=0):
        print("MINIMIZATION FAILED")
        print(minimization_command)    

In [6]:
def generate_population(input_pla_dictionary, population_filename):
    
    no_classifiers = 0
    
    with open(population_filename,"w") as POPULATION:

        for action in input_pla_dictionary.keys():
            with open(input_pla_dictionary[action], "r") as INPUT:
                str_pla = INPUT.read()
            entries = [x for x in str_pla.split("\n") if (not (x[:2] in [".i", ".o", ".e", ".p"])) and (x.strip()!="")]
            for entry in entries:
                condition,payoff = entry.split(" ")
                classifier = condition[:-1].strip().replace("-","#") + "\t" + str(action) + "\t"
                # print(condition[:-1].strip().replace("-","#"))
                if (payoff.strip())=="10":
                    classifier = classifier + "1000" + "\n"
                else:
                    classifier = classifier + "0" + "\n"
                no_classifiers = no_classifiers + 1
                POPULATION.write(classifier)
    return no_classifiers

In [ ]:
def compute_optimal_solution(prefix, n, binary_function, pla_directory="./pla",populations_directory="./populations/"):
    
    Path(populations_directory).mkdir(parents=True, exist_ok=True)
    Path(pla_directory).mkdir(parents=True, exist_ok=True)
    
    function_dictionary = {}
    for action in [0,1]:
        
        generate_action_table_v2(n=n, binary_function=binary_function, action=action, filename="%s/%s_%d.pla"%(pla_directory,prefix,action))

        # Uncomment this line to have a time evaluation using repeated runs
        # %timeit simplify_table("%s/%s_%d.pla"%(pla_directory,prefix,action), "%s/%s_%d_simplified.pla"%(pla_directory,prefix,action))
        simplify_table("%s/%s_%d.pla"%(pla_directory,prefix,action), "%s/%s_%d_simplified.pla"%(pla_directory,prefix,action))

        function_dictionary[action] = "%s/%s_%d_simplified.pla"%(pla_directory,prefix,action)
    
    no_classifiers = generate_population(input_pla_dictionary=function_dictionary, population_filename="%s/%s_classifiers.txt"%(populations_directory,prefix))
    
    print("%-8s\t# Classifiers = %d (%5.4f)\n\n"%(prefix,no_classifiers, no_classifiers/(2**(n+2))))    

## Generation of Optimal Solutions from TELO Paper
In this section we generate all the optimal populations for the paper: "A Comparison of Learning Classifier Systems’Rule Compaction Algorithms for Knowledge Visualization"

In [8]:
all_experiments = [('mux',multiplexer,[6,11,20]),('carry',carry,[6,8,10,12]),('majority_on',majority_on_action,[6,7,8,9,10,11,12])]

for label,binary_function,input_sizes in all_experiments:
    for input_size in input_sizes:
        prefix = label+str(input_size)
        compute_optimal_solution(prefix, input_size, binary_function)

mux6    # Classifiers = 16 (0.0625)


mux11   # Classifiers = 32 (0.0039)


mux20   # Classifiers = 64 (0.0000)


carry6  # Classifiers = 36 (0.1406)


carry8  # Classifiers = 76 (0.0742)


carry10 # Classifiers = 156 (0.0381)


carry12 # Classifiers = 316 (0.0193)


majority_on6# Classifiers = 70 (0.2734)


majority_on7# Classifiers = 140 (0.2734)


majority_on8# Classifiers = 252 (0.2461)


majority_on9# Classifiers = 504 (0.2461)


majority_on10# Classifiers = 924 (0.2256)


majority_on11# Classifiers = 1848 (0.2256)


majority_on12# Classifiers = 3432 (0.2095)




## Generate Single Functions

### Binary Multiplexer

In [21]:
prefix = "mp6"
n = 6
binary_function = mp

compute_optimal_solution(prefix, n, binary_function)

mp6     # Classifiers = 16 (0.0625)




In [22]:
prefix = "mp11"
n = 11
binary_function = mp

compute_optimal_solution(prefix, n, binary_function)

mp11    # Classifiers = 32 (0.0039)




In [23]:
prefix = "mp20"
n = 20
binary_function = mp

compute_optimal_solution(prefix, n, binary_function)

MINIMIZATION FAILED
../espresso-logic-master/bin/espresso -Dexact ./pla/mp20_0.pla > ./pla/mp20_0_simplified.pla
mp20    # Classifiers = 32 (0.0000)




### Eq(5,3)

In [ ]:
prefix = "eq53"
n = 5
binary_function = (lambda x: eq(x,5,3))

compute_optimal_solution(prefix, n, binary_function)

eq53    # Classifiers = 50 (0.3906)




### Majority On

In [ ]:
prefix = "majority5"
n = 5
binary_function = majority_on_action

compute_optimal_solution(prefix, n, binary_function)

majority5# Classifiers = 40 (0.3125)




### Carry 

In [24]:
prefix = "carry6"
n = 6
binary_function = carry

compute_optimal_solution(prefix, n, binary_function)

carry6  # Classifiers = 36 (0.1406)


